In [1]:
from acdesign.atmosphere import Atmosphere
from acdesign.performance.operating_point import OperatingPoint
from acdesign.airfoils.polar import UIUCPolars


clarky = UIUCPolars.local("CLARKYB")
sa7038 = UIUCPolars.local("SA7038")
e472 = UIUCPolars.local("E472")


In [2]:
from acdesign.performance.aero import AircraftAero, WingAero, FuseAero


wing = WingAero(5, 5 * 0.3, [clarky,sa7038],[0, 0.2, 1])
op = OperatingPoint(Atmosphere.alt(0), 5)

wing.stall(op)


{'S': 1.5,
 'c': 0.3,
 'AR': 16.666666666666668,
 'Cl': np.float64(1.2805714285714287),
 'Cd0': np.float64(0.03330126514497877),
 'k': 0.0272837045300392,
 'Cd': np.float64(0.07804280771801511),
 'stall': np.True_,
 'Cm': np.float64(-0.05139285590663371),
 're': np.float64(100959.73567637235)}

In [6]:
import plotly.express as px
from acdesign.performance.propulsion import Propeller, FactorMotor, ConstantPropeller
from acdesign.performance.aero import AircraftAero, WingAero, FuseAero
import numpy as np
import pandas as pd
import numpy.typing as npt
from scipy.optimize import minimize
import plotly.graph_objects as go

def solar_plane_aero(nrows, ncols):
    b = nrows * 0.14 + 1
    C = ncols * 0.13 + 0.07
    S = b * C

    fus_length = b/4

    return AircraftAero(
        WingAero(b, S, [clarky,sa7038],[0, 0.2222, 1]),
        WingAero(b*0.25, S*0.2, [e472],[0,1]),
        WingAero(b*0.15, S*0.1, [e472],[0,1]),
        FuseAero(fus_length, 0.05),
        0.02, fus_length * 0.75
    )


# now optimise to find maximum mass that can be supported by solar power
V = 11
mass = 1
propeller = ConstantPropeller(0.5, 3000)
motor = FactorMotor(0.65)


def preq(aero: AircraftAero, mass: float, V: float):

    op = OperatingPoint(Atmosphere.alt(0), V)
    drag = aero.calculate_drag(op, mass)

    rpm, torque = propeller.calculate(drag, op.V, op.atm.rho)
    
    return motor.calculate(rpm, torque) + 4 # add 4 W for peripherals


nrows = 20
ncols = 1
cell_power = 3 # watts per solar cell

solar_power = nrows * ncols * cell_power

aero = solar_plane_aero(nrows, ncols)


data = []
for mass in np.linspace(0.5, 3, 4):
    for V in np.linspace(5, 20, 20):
        data.append([mass, V, preq(aero, mass, V)])
    

df = pd.DataFrame(data, columns=["mass", "V", "preq"])


aero.wing.b, aero.wing.S, aero.wing.AR, aero.wing.smc, aero
#px.line(df, x="V", y="preq", color="mass")


(3.8000000000000003,
 0.7600000000000001,
 19.0,
 0.2,
 AircraftAero(wing=WingAero(b=3.8000000000000003, S=0.7600000000000001, polars=[<acdesign.airfoils.polar.UIUCPolars object at 0x7cfdec9692b0>, <acdesign.airfoils.polar.UIUCPolars object at 0x7cfdec9b4f50>], rib_locs=[0, 0.2222, 1]), tail=WingAero(b=0.9500000000000001, S=0.15200000000000002, polars=[<acdesign.airfoils.polar.UIUCPolars object at 0x7cfdec9b5090>], rib_locs=[0, 1]), fin=WingAero(b=0.5700000000000001, S=0.07600000000000001, polars=[<acdesign.airfoils.polar.UIUCPolars object at 0x7cfdec9b5090>], rib_locs=[0, 1]), fus=FuseAero(length=0.9500000000000001, diameter=0.05), pw=0.02, pt=0.7125, cd0_offset=0))

In [4]:

fig = go.Figure()

for mass in df["mass"].unique():
    fig.add_trace(
        go.Scatter(
            x=df[df["mass"]==mass]["V"],
            y=df[df["mass"]==mass]["preq"],
            mode="lines",
            name=f"mass={mass:.2f} kg"
        )
    )

fig.update_xaxes(title="Airspeed (m/s)")
fig.update_yaxes(title="Power Required (W)", range=[0,200])
fig.add_hline(y=solar_power, line_dash="dash", annotation_text="Solar Power Available", annotation_position="top left")
fig

In [5]:
res = {}

for cl in np.linspace(0.1, 1.5, 10):
    r = sa7038.lookup(np.linspace(10000, 400000, 100), np.full(100, cl))
    res[cl] = r.set_index("re").Cd
res = pd.concat(res, axis=1)

fig = go.Figure(data=go.Surface(z=res.values, x=res.columns, y=res.index))
fig
